In [1]:
!pip uninstall tensorflow
!pip install tensorflow==2.13.0

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.13.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.13.0
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (524.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.

In [2]:
import tensorflow
print(tensorflow.__version__)

2.13.0


In [3]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


In [4]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-09-04 05:20:59--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-09-04 05:20:59 (83.1 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [5]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys


In [6]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2024-09-04 05:20:59--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.207, 108.177.98.207, 74.125.197.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip.1’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2024-09-04 05:20:59 (81.5 MB/s) - ‘nlp_getting_started.zip.1’ saved [607343/607343]



In [7]:
# Turn .csv files into pandas DataFrame's
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [11]:
len(train_df),len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")


Target: 1 (real disaster)
Text:
Families to sue over Legionnaires: More than 40 families affected by the fatal outbreak of Legionnaires' disea... http://t.co/kkdX8zMV4G

---

Target: 0 (not real disaster)
Text:
GOP debate tonight but no Jon Stewart next week to obliterate them ?? #JonVoyage

---

Target: 1 (real disaster)
Text:
Cancers equate for around 25% of all deaths in #Blackpool. 

Kowing the signs could save your life: http://t.co/11dVqjVXPo
#BeClearOnCancer

---

Target: 0 (not real disaster)
Text:
this is about to be a bomb ass firework picture http://t.co/lr4BTvuEoM

---

Target: 0 (not real disaster)
Text:
@cinla1964 @windowgatribble The Contrasts of Foreboding destruction enhanced by Expansive divisions of color saturation contrast and hue!

---



In [13]:
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                      train_df_shuffled["target"].to_numpy(),
                                                                      test_size=0.1,
                                                                      random_state=42)

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

max_length = round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_vocab_length = 1000

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [15]:
text_vectorizer.adapt(train_sentences)

In [16]:
sample = "There's a flood in my street!"
text_vectorizer([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [17]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
@PipRhys I predict a riot.      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  1,   8,   1,   3, 416,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 1000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['reported', 'r', 'pray', 'playlist', 'patience']


In [19]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             name="embedding_1")
embedding

In [20]:
train_sentences_vectorized = text_vectorizer(train_sentences)
train_sentences_vectorized[:10]

<tf.Tensor: shape=(10, 15), dtype=int64, numpy=
array([[  1,   1,   8, 387,  43,   1, 978,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1, 209, 485,  18,   1,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,   1, 850, 236,   1,   1,   1,   1,   1,   1,   1,  92,  75,
          1,   1],
       [  1,   1,   1,  32,   1,   4, 201, 277, 319,   1,   2,   1,   0,
          0,   0],
       [  1, 653,  12,   7,   8, 517,   1,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,   1, 196, 252, 293,   2, 352, 900,   1,  69, 509,   4,   2,
          1,   1],
       [305,   2, 268,   1,   1,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [258,   1,  20, 372,   1,   1,   4,  50,   1,  80, 373,   1,   1,
          0,   0],
       [  1,   1,  45,  38,   2, 288, 472,   1,   9,  15, 169, 104,   5,
         21,   1],
       [  1,   1,   4,   1, 362,   6,   1,   1,   1, 961,   1,   6,   1,
          1,   0]])>

In [21]:
train_sentences_embed = embedding(train_sentences_vectorized)
train_sentences_embed[0]

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.03290213,  0.01717793,  0.04523022, ...,  0.03791467,
         0.0457829 , -0.02148349],
       [ 0.03290213,  0.01717793,  0.04523022, ...,  0.03791467,
         0.0457829 , -0.02148349],
       [-0.03539368, -0.01815573,  0.04498551, ..., -0.03286548,
        -0.04402261,  0.04406041],
       ...,
       [ 0.02227363, -0.00234227,  0.03465934, ...,  0.04720246,
         0.04614494,  0.04681457],
       [ 0.02227363, -0.00234227,  0.03465934, ...,  0.04720246,
         0.04614494,  0.04681457],
       [ 0.02227363, -0.00234227,  0.03465934, ...,  0.04720246,
         0.04614494,  0.04681457]], dtype=float32)>

# Baseline Model

## Sklearn's Multi Dimensional Naive Bayes

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB())
])

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [24]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred)
  model_0_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_0_results

baseline_results = calculate_results(y_true=val_labels,
                                    y_pred=model_0.predict(val_sentences))
baseline_results

{'accuracy': 79.26509186351706,
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678]),
 'f1': array([0.83010753, 0.73400673])}

# A Simple Dense Model

In [25]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [26]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [27]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           128000    
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 128129 (500.50 KB)
Trainable params: 12

In [28]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [29]:
train_sentences.shape,train_labels

((6851,), array([0, 0, 1, ..., 1, 1, 0]))

In [30]:
model_1_history = model_1.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                           experiment_name = "mdoel_1_dense_model")])

Saving TensorBoard log files to: model_logs/mdoel_1_dense_model/20240904-052102
Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.6232 - accuracy: 0.6684 - val_loss: 0.5607 - val_accuracy: 0.7297
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4925 - accuracy: 0.7776 - val_loss: 0.4981 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.4377 - accuracy: 0.8060 - val_loss: 0.4858 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4109 - accuracy: 0.8186 - val_loss: 0.4878 - val_accuracy: 0.7795
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3942 - accuracy: 0.8215 - val_loss: 0.5011 - val_accuracy: 0.7730


In [31]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 1s 12ms/step - loss: 0.5011 - accuracy: 0.7730


[0.501095175743103, 0.7729659080505371]

In [32]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 1s 8ms/step


(762, 1)

In [33]:
model_1_pred = tf.squeeze(tf.round(model_1_pred_probs))
model_1_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [34]:
model_1_results = calculate_results(val_labels,model_1_pred)
model_1_results

{'accuracy': 77.29658792650919,
 'precision': array([0.73489279, 0.85140562]),
 'recall': array([0.91062802, 0.6091954 ]),
 'f1': array([0.81337648, 0.71021776])}

In [35]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': array([0.74806202, 0.88617886]),
 'recall': array([0.93236715, 0.62643678]),
 'f1': array([0.83010753, 0.73400673])}

## RNN neural network

In [36]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64,return_sequences=True)(x)
x = layers.GlobalAveragePooling1D()(x)

outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="LSTM")


In [37]:
model_2.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           128000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            49408     
                                                                 
 global_average_pooling1d_1  (None, 64)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_1 (Dense)             (None, 1)                 65     

In [38]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                           experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20240904-052128
Epoch 1/5
215/215 [==============================] - 27s 71ms/step - loss: 0.4144 - accuracy: 0.8205 - val_loss: 0.4810 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 12s 56ms/step - loss: 0.3864 - accuracy: 0.8292 - val_loss: 0.5261 - val_accuracy: 0.7664
Epoch 3/5
215/215 [==============================] - 11s 50ms/step - loss: 0.3800 - accuracy: 0.8313 - val_loss: 0.5000 - val_accuracy: 0.7638
Epoch 4/5
215/215 [==============================] - 11s 51ms/step - loss: 0.3730 - accuracy: 0.8336 - val_loss: 0.5043 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 10s 45ms/step - loss: 0.3650 - accuracy: 0.8383 - val_loss: 0.4991 - val_accuracy: 0.7717


In [39]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[0.4535628 ],
       [0.71181643],
       [0.9468407 ],
       [0.1071663 ],
       [0.15402089],
       [0.99718416],
       [0.7439522 ],
       [0.8618321 ],
       [0.6320737 ],
       [0.17750028]], dtype=float32)

In [40]:
model_2_pred = tf.squeeze(tf.round(model_2_pred_probs))
model_2_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [82]:
model_2_results = calculate_results(val_labels,model_2_pred)
model_2_results

{'accuracy': 77.16535433070865,
 'precision': array([0.74793388, 0.81294964]),
 'recall': array([0.87439614, 0.64942529]),
 'f1': array([0.80623608, 0.72204473])}

## GRU Network

In [42]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name="GRU")


In [43]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           128000    
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 165313 (645.75 KB)
Trainable params: 165313 (645.75 KB)
Non-trainable params: 0 (0.00 Byte)
_______________________

In [44]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [45]:
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                         "model_3_GRU")])


Saving TensorBoard log files to: model_logs/model_3_GRU/20240904-052302
Epoch 1/5
215/215 [==============================] - 9s 21ms/step - loss: 0.4032 - accuracy: 0.8170 - val_loss: 0.4917 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.3643 - accuracy: 0.8429 - val_loss: 0.5091 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 7s 31ms/step - loss: 0.3493 - accuracy: 0.8495 - val_loss: 0.5502 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.3337 - accuracy: 0.8616 - val_loss: 0.5575 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 6s 26ms/step - loss: 0.3154 - accuracy: 0.8720 - val_loss: 0.5612 - val_accuracy: 0.7743


In [46]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[0.6791244 ],
       [0.77889615],
       [0.955454  ],
       [0.05418659],
       [0.0396824 ],
       [0.996092  ],
       [0.81555307],
       [0.8361997 ],
       [0.5738378 ],
       [0.100126  ]], dtype=float32)

In [47]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_3_results = calculate_results(val_labels,model_3_preds)
model_3_results

{'accuracy': 77.42782152230971,
 'precision': array([0.742     , 0.83587786]),
 'recall': array([0.89613527, 0.62931034]),
 'f1': array([0.81181619, 0.71803279])}

# Bidirectional RNN
### Previously discussed RNN's move from right to left only
### This moves in both directions

In [49]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="bidirectional_RNN")


In [50]:
model_4.summary()

Model: "bidirectional_RNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           128000    
                                                                 
 bidirectional (Bidirection  (None, 15, 128)           98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                                 

In [51]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [85]:
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences,val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                         "model_4_bidirectional_RNN")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional_RNN/20240904-061053
Epoch 1/5
215/215 [==============================] - 12s 56ms/step - loss: 0.2011 - accuracy: 0.9161 - val_loss: 0.8095 - val_accuracy: 0.7441
Epoch 2/5
215/215 [==============================] - 11s 53ms/step - loss: 0.1837 - accuracy: 0.9277 - val_loss: 0.8955 - val_accuracy: 0.7323
Epoch 3/5
215/215 [==============================] - 11s 51ms/step - loss: 0.1606 - accuracy: 0.9324 - val_loss: 1.0347 - val_accuracy: 0.7283
Epoch 4/5
215/215 [==============================] - 22s 105ms/step - loss: 0.1444 - accuracy: 0.9396 - val_loss: 1.1344 - val_accuracy: 0.7388
Epoch 5/5
215/215 [==============================] - 12s 57ms/step - loss: 0.1372 - accuracy: 0.9451 - val_loss: 1.1042 - val_accuracy: 0.7388


In [53]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 3s 23ms/step


array([[0.91047764],
       [0.82470024],
       [0.9988964 ],
       [0.0587191 ],
       [0.08688036],
       [0.9992051 ],
       [0.99722475],
       [0.8430236 ],
       [0.666811  ],
       [0.1213942 ]], dtype=float32)

In [54]:
model_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [55]:
model_4_results = calculate_results(val_labels,model_preds)
model_4_results

{'accuracy': 74.01574803149606,
 'precision': array([0.73376623, 0.75      ]),
 'recall': array([0.81884058, 0.64655172]),
 'f1': array([0.7739726 , 0.69444444])}

# CNN for NLP

In [56]:
from tensorflow.keras import layers

input = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(filters=64,kernel_size=5,activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
output = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_CNN")

In [57]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
model_5.summary()

Model: "model_5_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 15)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           128000    
                                                                 
 bidirectional (Bidirection  (None, 15, 128)           98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               74496     
 onal)                                                           
                                                       

In [59]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20240904-052510
215/215 [==============================] - 30s 78ms/step - loss: 0.2040 - accuracy: 0.9168 - val_loss: 0.8146 - val_accuracy: 0.7533


In [60]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 2s 11ms/step


array([[0.45131588],
       [0.7854062 ],
       [0.9998911 ],
       [0.06413919],
       [0.08335914],
       [0.9999079 ],
       [0.9895733 ],
       [0.99368745],
       [0.70236456],
       [0.15406375]], dtype=float32)

In [61]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [62]:
model_5_results = calculate_results(val_labels,model_5_preds)
model_5_results

{'accuracy': 75.32808398950131,
 'precision': array([0.74353448, 0.76845638]),
 'recall': array([0.83333333, 0.65804598]),
 'f1': array([0.78587699, 0.70897833])}

# Tensoflow Hub Pretrained Feature extractor

In [63]:
import tensorflow_hub as hub

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2")
embed(['hello'])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-3.52459848e-02, -7.76301622e-02,  2.36061122e-02,
         3.80813740e-02,  7.40554929e-02,  3.58151607e-02,
         3.92020792e-02, -5.65796681e-02,  6.96732700e-02,
         5.97005710e-02,  6.77300170e-02,  4.32087928e-02,
         1.80188715e-02,  8.83076712e-02, -4.01138794e-03,
        -3.81293334e-02, -2.68758852e-02,  2.04251651e-02,
         2.95902174e-02, -5.37502766e-02, -5.91573641e-02,
         2.84411479e-02, -5.14547294e-03,  4.24177125e-02,
        -6.75308406e-02,  2.97567025e-02,  3.11530028e-02,
        -6.78873733e-02, -1.51054272e-02, -2.32834108e-02,
         5.03614992e-02,  2.93063615e-02,  7.68465269e-03,
        -5.18777147e-02, -4.17641290e-02,  9.24949534e-03,
        -8.15916434e-03,  6.25761691e-03,  4.40120474e-02,
         2.36562993e-02, -3.87538671e-02,  3.24229244e-03,
         1.32664701e-03, -4.83405106e-02,  7.99960829e-03,
         4.68025133e-02, -5.20302691e-02, -5.56475893e-02,
      

In [64]:
sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/2",
                                        input_shape = [],
                                        dtype =tf.string,
                                        trainable=False,
                                        name="USE")


In [65]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)


2.13.0


In [66]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64),
    layers.Dense(1,activation='sigmoid')
    ],name = "model_6_USE"
)

In [67]:
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [68]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [69]:
model_6_history = model_6.fit(train_sentences,train_labels,
                              epochs=5,
                              validation_data = (val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_sentence_encodder")])

Saving TensorBoard log files to: model_logs/tf_sentence_encodder/20240904-052619
Epoch 1/5
215/215 [==============================] - 8s 19ms/step - loss: 0.4881 - accuracy: 0.7881 - val_loss: 0.4507 - val_accuracy: 0.8031
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4156 - accuracy: 0.8130 - val_loss: 0.4539 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4067 - accuracy: 0.8210 - val_loss: 0.4393 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.4027 - accuracy: 0.8227 - val_loss: 0.4411 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3997 - accuracy: 0.8215 - val_loss: 0.4436 - val_accuracy: 0.8123


In [70]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 18ms/step


array([[0.250417  ],
       [0.8130587 ],
       [0.987471  ],
       [0.12634072],
       [0.818433  ],
       [0.7067421 ],
       [0.9758033 ],
       [0.97445154],
       [0.9106098 ],
       [0.12339913]], dtype=float32)

In [71]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [72]:
model_6_results = calculate_results(val_labels,model_6_preds)
model_6_results

{'accuracy': 81.23359580052494,
 'precision': array([0.799117  , 0.83171521]),
 'recall': array([0.87439614, 0.73850575]),
 'f1': array([0.83506344, 0.78234399])}

# TF Hub Pretrained USE with 10% dataset

In [73]:
train_10 = train_df_shuffled[["text",'target']].sample(frac=0.1,random_state=42)
train_sentences_10 = train_10['text'].to_list()
train_labels_10 = train_10['target'].to_list()

In [74]:
train_10["target"].value_counts()

,count
target,
0,413
1,348


In [75]:
model_7 = tf.keras.models.clone_model(model_6)

model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_7.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_5 (Dense)             (None, 64)                32832     
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256830721 (979.73 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 256797824 (979.61 MB)
_________________________________________________________________


In [86]:
model_7_history = model_7.fit(train_sentences_10,train_labels_10,
                              epochs=5,validation_data=(
                                  val_sentences,val_labels
                              ))

Epoch 1/5
24/24 [==============================] - 1s 32ms/step - loss: 0.3616 - accuracy: 0.8489 - val_loss: 0.3475 - val_accuracy: 0.8517
Epoch 2/5
24/24 [==============================] - 1s 57ms/step - loss: 0.3483 - accuracy: 0.8541 - val_loss: 0.3369 - val_accuracy: 0.8556
Epoch 3/5
24/24 [==============================] - 1s 56ms/step - loss: 0.3397 - accuracy: 0.8555 - val_loss: 0.3269 - val_accuracy: 0.8609
Epoch 4/5
24/24 [==============================] - 1s 52ms/step - loss: 0.3290 - accuracy: 0.8633 - val_loss: 0.3178 - val_accuracy: 0.8648
Epoch 5/5
24/24 [==============================] - 1s 53ms/step - loss: 0.3210 - accuracy: 0.8633 - val_loss: 0.3096 - val_accuracy: 0.8675


In [77]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 13ms/step


array([[0.24006866],
       [0.8321479 ],
       [0.95216984],
       [0.29088616],
       [0.817812  ],
       [0.8454162 ],
       [0.9207153 ],
       [0.9703781 ],
       [0.8645808 ],
       [0.04744916]], dtype=float32)

In [78]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [79]:
model_7_results = calculate_results(val_labels,model_7_preds)
model_7_results

{'accuracy': 82.1522309711286,
 'precision': array([0.82938389, 0.81176471]),
 'recall': array([0.84541063, 0.79310345]),
 'f1': array([0.83732057, 0.80232558])}

In [83]:
all_model_results = pd.DataFrame({"0_baseline": baseline_results,
                                  "1_simple_dense": model_1_results,
                                  "2_lstm": model_2_results,
                                  "3_gru": model_3_results,
                                  "4_bidirectional": model_4_results,
                                  "5_conv1d": model_5_results,
                                  "6_tf_hub_use_encoder": model_6_results,
                                  "7_tf_hub_use_encoder_10_percent": model_7_results})
all_model_results = all_model_results.transpose()
all_model_results

,accuracy,precision,recall,f1
0_baseline,79.265092,"[0.748062015503876, 0.8861788617886179]","[0.9323671497584541, 0.6264367816091954]","[0.8301075268817204, 0.734006734006734]"
1_simple_dense,77.296588,"[0.7348927875243665, 0.8514056224899599]","[0.9106280193236715, 0.6091954022988506]","[0.8133764832793959, 0.7102177554438861]"
2_lstm,77.165354,"[0.7479338842975206, 0.8129496402877698]","[0.8743961352657005, 0.6494252873563219]","[0.8062360801781737, 0.7220447284345047]"
3_gru,77.427822,"[0.742, 0.8358778625954199]","[0.8961352657004831, 0.6293103448275862]","[0.811816192560175, 0.7180327868852459]"
4_bidirectional,74.015748,"[0.7337662337662337, 0.75]","[0.8188405797101449, 0.646551724137931]","[0.773972602739726, 0.6944444444444444]"
5_conv1d,75.328084,"[0.7435344827586207, 0.7684563758389261]","[0.8333333333333334, 0.6580459770114943]","[0.785876993166287, 0.7089783281733746]"
6_tf_hub_use_encoder,81.233596,"[0.7991169977924945, 0.8317152103559871]","[0.8743961352657005, 0.7385057471264368]","[0.8350634371395617, 0.7823439878234398]"
7_tf_hub_use_encoder_10_percent,82.152231,"[0.8293838862559242, 0.8117647058823529]","[0.8454106280193237, 0.7931034482758621]","[0.8373205741626794, 0.8023255813953488]"


In [88]:
!tensorboard dev upload --logdir ./model_logs/ --name "NLP Modelling" --one_shot

2024-09-04 06:13:59.363998: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

***** TensorBoard Uploader *****

This will upload your TensorBoard logs to https://tensorboard.dev/ from
the following directory:

./model_logs/

This TensorBoard will be visible to everyone. Do not upload sensitive
data.

Your use of this service is subject to Google's Terms of Service
<https://policies.google.com/terms> and Privacy Policy
<https://policies.google.com/privacy>, and TensorBoard.dev's Terms of Service
<https://tensorboard.dev/policy/terms/>.

This notice will not be shown again while you are logged into the uploader.
To log out, run `tensorboard dev auth revoke`.

Continue? (yes/NO) yes

Traceback (most recent call last):
  File "/usr/local/bin/tensorboard", line 8, in <module>
    sys.exit(run_main())
  File "/usr/local/lib/python3.10/dist-packages/tensorboard/main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.